In [1]:
# from image_processing import *
import dlib
import cv2
import numpy as np

# Load the detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
def extract_eye_region(image, landmarks, eye_points, buffer= 0):
    # Extract the coordinates of the eye points
    region = np.array([(landmarks.part(point).x, landmarks.part(point).y) for point in eye_points])
    # Create a mask with zeros
    height, width = image.shape[:2]
    mask = np.zeros((height, width), np.uint8)
    # Fill the mask with the polygon defined by the eye points
    cv2.fillPoly(mask, [region], 255)
    # Bitwise AND operation to isolate the eye region
    eye = cv2.bitwise_and(image, image, mask=mask)
    # Cropping the eye region
    (min_x, min_y) = np.min(region, axis=0)
    (max_x, max_y) = np.max(region, axis=0)
    min_x = max(min_x - buffer, 0)
    min_y = max(min_y - buffer, 0)
    max_x = min(max_x + buffer, width)
    max_y = min(max_y + buffer, height)
    cropped_eye = eye[min_y:max_y, min_x:max_x]
    return cropped_eye, (min_x, min_y, max_x, max_y)


In [59]:
import cv2

def enhance_image_resolution(image):
    # Load the super-resolution model
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"  # Change to the path of the model
    sr.readModel(path)
    sr.setModel("edsr", 4)  # You can change the model and scale as needed

    # Enhance the resolution of the image
    enhanced_image = sr.upsample(image)
    return enhanced_image


In [60]:
def visualize_image(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [104]:
def detect_pupil(eye_image):
    # Enhance resolution
    eye_image = enhance_image_resolution(eye_image)

    gray = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    visualize_image("Grayscale Eye", gray)  # Visualize grayscale eye

    gray = cv2.equalizeHist(gray)  # Histogram Equalization
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)  # Gaussian Blur
    visualize_image("Blurred Eye", blurred)  # Visualize blurred eye

    # Thresholding
    _, thresholded = cv2.threshold(blurred, 40, 255, cv2.THRESH_BINARY_INV)
    visualize_image("Thresholded Eye", thresholded)  # Visualize thresholded eye

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    opened = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)
    visualize_image("Opened Eye", opened)  # Visualize opened eye

    # Find contours
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and filter
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Filter out edge contours
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x > 0 and y > 0 and x + w < eye_image.shape[1] and y + h < eye_image.shape[0]:
            filtered_contours.append(contour)

    # Visualize contours
    contour_image = eye_image.copy()
    cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), 2)
    visualize_image("Contours", contour_image)

    for contour in filtered_contours:
        # Rest of the code...

        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)

        # Check if perimeter is zero to avoid division by zero
        if perimeter == 0:
            print("Perimeter is zero")
            continue

        # Check for circularity
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        print(f'Circularity: {circularity}')
        if circularity < 0.5:
            continue

        # Found a good pupil candidate
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.circle(eye_image, (cX, cY), 7, (255, 255, 255), -1)
            visualize_image("Pupil", eye_image)
            return (cX, cY), contour

    print("No suitable pupil found")
    return None, None

In [157]:
def detect_pupil(eye_image):
    # Enhance resolution
    eye_image = enhance_image_resolution(eye_image)

    # Convert to grayscale
    gray = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    visualize_image("gray", gray)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_equalized = clahe.apply(gray)
    visualize_image("clahe_equalized", clahe_equalized)

    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(clahe_equalized, (7, 7), 0)

    # Apply Adaptive Thresholding
    adaptive_thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV, 11, 2)

    # Perform Edge Detection using Canny
    edges = cv2.Canny(adaptive_thresh, 50, 150)#
    visualize_image("edges", edges)

    # Use Hough Circle Transform to find circles
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
                               param1=30, param2=20, minRadius=12, maxRadius=30)
    print(circles)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(eye_image, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(eye_image, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
        return eye_image, circles

    print("No suitable pupil found")
    return eye_image, None

In [158]:
def pre_process_image(image):
    # Initialize variables
    left_eye_info = right_eye_info = left_eye_bbox = right_eye_bbox = None

    dlib_faces = detector(image)
    processed_data = []
    for dlib_face in dlib_faces:
        shape = predictor(image, dlib_face)

        for (i, (start, end)) in enumerate([(36,42), (42,48)]):
            eye_landmarks = [(shape.part(point).x, shape.part(point).y) for point in range(start, end)]
            eye_image, (eye_min_x, eye_min_y, eye_max_x, eye_max_y) = extract_eye_region(image, shape, range(start, end))
  
            # Call the new detect_pupil function
            result_eye_image, circles = detect_pupil(eye_image)
            
            # After detecting the pupil in the cropped eye image:
            if circles is not None:
                # Select the largest circle as the pupil
                largest_circle = max(circles, key=lambda c: c[2])
                x, y, r = largest_circle
                pupil_center = (int(x/4), int(y/4))


                # Create a contour from the circle for compatibility
                pupil_contour = cv2.ellipse2Poly((x, y), (r, r), 0, 0, 360, 1)

                # Transform the pupil center coordinates to the global space of the original image
                pupil_center_global = (pupil_center[0] + eye_min_x, pupil_center[1] + eye_min_y)

                # Draw contours and center on the original image
                # Draw the pupils 
                cv2.circle(image, pupil_center_global, 1, (0, 255, 0), 2)
                # Draw the contour
                

                cv2.imshow("image", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


                bounding_box = (eye_min_x, eye_min_y, eye_max_x - eye_min_x, eye_max_y - eye_min_y)
                eye_data = {
                    'eye_position': 'left' if i == 0 else 'right',
                    'pupil_center': pupil_center_global,
                    'bounding_box': bounding_box,
                }
                processed_data.append(eye_data)

    # Processed data for each eye
    for eye_data in processed_data:
        if eye_data['eye_position'] == 'left':
            left_eye_info = eye_data['pupil_center']
            left_eye_bbox = eye_data['bounding_box']
        else:
            right_eye_info = eye_data['pupil_center']
            right_eye_bbox = eye_data['bounding_box']

    # Check if any eye information was detected
    if left_eye_info is None and right_eye_info is None:
        print("No eye information detected")
        return None

    # Returning the processed data along with the dlib shape for further processing if needed
    return processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape

In [159]:
image = 'data/Naia/calibration_images/Naia_07779f08-1c1f-490f-9d26-7786c43aca1d.png'
pre_process_image(cv2.imread(image))
# # Load your image here
# image = cv2.imread(image)
# eye_image = extract_eye_region(image)


[[[74.5 17.5 20.5]]]
[[[67.5 18.5 15.8]]]


([{'eye_position': 'left',
   'pupil_center': (311, 165),
   'bounding_box': (293, 161, 36, 10)},
  {'eye_position': 'right',
   'pupil_center': (395, 169),
   'bounding_box': (378, 165, 33, 10)}],
 (311, 165),
 (395, 169),
 (293, 161, 36, 10),
 (378, 165, 33, 10),
 <_dlib_pybind11.full_object_detection at 0x234eef0ee30>)

In [160]:
image1 = 'data/Will/eye_gaze_images/Will_0c426e4f-72ac-479c-a1e6-3d9df004aebb.png'
pre_process_image(cv2.imread(image1))

[[[82.5 21.5 19.4]]]
[[[49.5 15.5 21.7]]]


([{'eye_position': 'left',
   'pupil_center': (293, 233),
   'bounding_box': (273, 228, 31, 11)},
  {'eye_position': 'right',
   'pupil_center': (363, 236),
   'bounding_box': (351, 232, 28, 10)}],
 (293, 233),
 (363, 236),
 (273, 228, 31, 11),
 (351, 232, 28, 10),
 <_dlib_pybind11.full_object_detection at 0x234ee6e6070>)

In [161]:
image2 = 'data/muzzy/calibration_images/muzzy_0194b294-0505-498e-afe8-837a4e1b5cf6.png'
pre_process_image(cv2.imread(image2))

[[[84.5 17.5 13.6]]]
[[[50.5 24.5 20.5]]]


([{'eye_position': 'left',
   'pupil_center': (296, 223),
   'bounding_box': (275, 219, 41, 12)},
  {'eye_position': 'right',
   'pupil_center': (387, 227),
   'bounding_box': (375, 221, 41, 12)}],
 (296, 223),
 (387, 227),
 (275, 219, 41, 12),
 (375, 221, 41, 12),
 <_dlib_pybind11.full_object_detection at 0x2348634f8f0>)